# Find bars nearby soccer stadiums located in Germany

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from geopedestrian.services import solve_walking
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory

In [2]:
class LocalClient(GeoRapidClient):
    
    def __init__(self, url, auth_headers):
        pass
    
    @property
    def auth_headers(self):
        return None
    
    @property
    def url(self):
        return 'http://localhost:7071/api'

## Location of Olympiastadion, Berlin

In [3]:
lat, lon = 52.514610, 13.239665

In [4]:
client = LocalClient(None, None)

In [9]:
walking_geojson = solve_walking(client, lat, lon)
walking_fset = FeatureSet.from_geojson(walking_geojson)
walking_fset.sdf

,ObjectID,FacilityID,Name,FromBreak,ToBreak,Shape_Length,Shape_Area,SHAPE
0,1,1,Location 1 : 10 - 15,10,15,0.15438372101428938,0.00016588132385536136,"{""rings"": [[[13.241466144000071, 52.5223607870..."
1,2,1,Location 1 : 5 - 10,5,10,0.09693572565674625,8.546242622503289e-05,"{""rings"": [[[13.241915299000027, 52.5194412760..."
2,3,1,Location 1 : 0 - 5,0,5,0.028781889442187052,2.7108911552395858e-05,"{""rings"": [[[13.241466144000071, 52.5165217650..."


In [7]:
gis = GIS()

In [8]:
berlin_map = gis.map('Berlin, Germany')

In [10]:
walking_fset.sdf.spatial.plot(berlin_map)
berlin_map

MapView(layout=Layout(height='400px', width='100%'))